In [1]:
# ! git clone https://github.com/vonum/multi-prophet.git

In [2]:
# !pip install pystan==2.19.1.1
# !pip install fbprophet
import sys
sys.path.append('multi-prophet')
from multi_prophet import MultiProphet

In [3]:
import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from prophet import Prophet

In [5]:
df = pd.read_csv('monica_weather_2011-2016.csv')
df["date"] = pd.to_datetime(df["de-date"], format='%d.%m.%Y')

In [6]:
df = df.rename(columns={"date": "ds"})

In [7]:
m = MultiProphet(columns=["tavg", "tmin", "tmax", "wind", "globrad", "precip", "relhumid"])
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [8]:
future = m.make_future_dataframe(periods=365, include_history=False)
forecast = m.predict(future)

In [9]:
test = pd.read_csv('reanalysis_2017.csv')

In [10]:
test['tavg'] = forecast['tavg'].yhat.to_numpy()
test['tmin'] = forecast['tmin'].yhat.to_numpy()
test['tmax'] = forecast['tmax'].yhat.to_numpy()
test['wind'] = forecast['wind'].yhat.to_numpy()
test['globrad'] = forecast['globrad'].yhat.to_numpy()
test['precip'] = forecast['precip'].yhat.to_numpy()
test['relhumid'] = forecast['relhumid'].yhat.to_numpy()

In [12]:
test.to_csv('multi-prophet_2017.csv', index=False)